# Introduction

In [2]:
# Creating SQL environment

%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: @factbook.db'

# Available Tables in The Database

In [3]:
%%sql

SELECT * FROM sqlite_master;

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Data Overview

In [4]:
%%sql

select * from facts
limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Column Descriptions

Here are the descriptions for some of the columns:
- name - The name of the country.
- area- The country's total area (both land and water).
- area_land - The country's land area in [square kilometers](https://www.cia.gov/library/publications/the-world-factbook/rankorder/2147rank.html).
- area_water - The country's waterarea in square kilometers.
- population - The country's population.
- population_growth- The country's population growth as a percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.

Let's start by calculating some summary statistics and look for any outlier countries.

# Finding Outliers

In [5]:
%%sql

SELECT min(population), min(population_growth), max(population), max(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


min(population),min(population_growth),max(population),max(population_growth)
0,0.0,7256490011,4.02


So, we can something unusual from the summary statistics:
- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

**Executing a query that returns the countrie(s) with the minimum population.**

**Using sub queries**

In [6]:
%%sql

SELECT * FROM facts
WHERE population = (SELECT min(population) FROM facts);   

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


**Executing a query that returns the countrie(s) with the maximum population.**

In [7]:
%%sql

SELECT * FROM facts
WHERE population = (SELECT max(population) FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


-  <b>It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world. </b>

In [8]:
%%sql

SELECT min(population), min(population_growth), max(population), max(population_growth)
FROM facts
WHERE population != 7256490011;

 * sqlite:///factbook.db
Done.


min(population),min(population_growth),max(population),max(population_growth)
0,0.0,1367485388,4.02


-  So, we found out that there is a country with 1367485388 population.

**Exploring the average values for population and area**

In [9]:
%%sql

SELECT avg(population), avg(area) FROM facts
WHERE population != 7256490011;

 * sqlite:///factbook.db
Done.


avg(population),avg(area)
32242666.56846473,582949.8523206752


-   **From the table above, we could see that the average population is approximately 32 million and average area is also 555 thousand square kilometers.**

**Let's explore the countries that are densely populated**

We'll identify countries that have:
- Above average values for population.
- Below average values for area.


In [10]:
%%sql
SELECT * 
FROM facts
WHERE (population > (SELECT avg(population) FROM facts WHERE population <> 7256490011) AND 
       area < (SELECT avg(area) FROM facts WHERE population <> 7256490011));

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
88,ke,Kenya,580367,569140,11227,45925301,1.93,26.4,6.89,0.22
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46


**Let's figure out**
-  What country has the most people?
-  What country has the highest growth rate?
 


In [11]:
%%sql

SELECT * FROM facts
WHERE population = (SELECT max(population) 
                    FROM facts 
                    WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


-    **China has the highest number of people**

In [12]:
%%sql

SELECT * FROM facts
WHERE population_growth = (SELECT max(population_growth) 
                    FROM facts 
                    WHERE name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


-   **South Sudan has highest number of population growth**

**Let's explore the below queries**
-  Which countries have the highest ratios of water to land?
-  Which countries have more water than land?

In [13]:
%%sql

SELECT name, area_land, area_water, 
        ROUND(CAST(area_water AS float)/CAST(area_land AS float) , 4) AS water_land 
FROM facts
ORDER BY water_land DESC
LIMIT 10; 

 * sqlite:///factbook.db
Done.


name,area_land,area_water,water_land
British Indian Ocean Territory,60,54340,905.6667
Virgin Islands,346,1564,4.5202
Puerto Rico,8870,4921,0.5548
"Bahamas, The",10010,3870,0.3866
Guinea-Bissau,28120,8005,0.2847
Malawi,94080,24404,0.2594
Netherlands,33893,7650,0.2257
Uganda,197100,43938,0.2229
Eritrea,101000,16600,0.1644
Liberia,96320,15049,0.1562


-   **Britis indian ocean territory has the highest water to land ratio followed by Virgin Islands, Puerto Rico etc**

In [14]:
%%sql

SELECT * FROM facts
WHERE area_water > area_land
ORDER BY area_water
LIMIT 5; 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None


-    **We wanted to top countries which have more water area than land but theb query only shows two. That means in the whole world only two places have more water area than land**

**Exploring countries with increase population growth next year**

In [15]:
%%sql

SELECT name, area, population, population_growth FROM facts
ORDER BY population_growth DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area,population,population_growth
South Sudan,644329,12042910,4.02
Malawi,118484,17964697,3.32
Burundi,27830,10742276,3.28
Niger,None,18045729,3.25
Uganda,241038,37101745,3.24
Qatar,11586,2194817,3.07
Burkina Faso,274200,18931686,3.03
Mali,1240192,16955536,2.98
Cook Islands,236,9838,2.95
Iraq,438317,37056169,2.93


-   **The above are the possible countries which would add more people to their population**

**Exploring which countries have a higher death rate than birth rate?**

In [16]:
%%sql

SELECT * FROM facts
WHERE death_rate > birth_rate
ORDER BY death_rate DESC
LIMIT 7;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7


-    **The above are the countries which have high death rate. The dataset is arranged from high death rate to low death rate**

**Exploring what countries have the highest population/area ratio.**

In [17]:
%%sql

SELECT name, area, population, CAST(population AS float) / CAST(area AS float) AS population_area
FROM facts
ORDER BY population_area DESC
LIMIT 15;


 * sqlite:///factbook.db
Done.


name,area,population,population_area
Macau,28,592731,21168.964285714286
Monaco,2,30535,15267.5
Singapore,697,5674472,8141.279770444763
Hong Kong,1108,7141106,6445.041516245487
Gaza Strip,360,1869055,5191.819444444444
Gibraltar,6,29258,4876.333333333333
Bahrain,760,1346613,1771.8592105263158
Maldives,298,393253,1319.6409395973155
Malta,316,413965,1310.01582278481
Bermuda,54,70196,1299.9259259259259


# Conclusion

**In this project we worked the basic queries to get some key information out of the dataset.**

## Thank you for spending time on my project.